## 2.1 urllib

In [1]:
import socket
from urllib import request
from urllib import parse
from urllib import error

In [2]:
with request.urlopen('https://api.douban.com/v2/book/2224879') as f:
    data = f.read()
    print('Status: {0} {1}'.format(f.status, f.reason))
    for k, v in f.getheaders():
        print('{0}: {1}'.format(k, v))
    print('Data:\n', data.decode('utf-8'))

Status: 200 OK
Date: Mon, 05 Nov 2018 05:02:06 GMT
Content-Type: application/json; charset=utf-8
Content-Length: 1179
Connection: close
Vary: Accept-Encoding
X-Ratelimit-Remaining2: 99
X-Ratelimit-Limit2: 100
Expires: Sun, 1 Jan 2006 01:00:00 GMT
Pragma: no-cache
Cache-Control: must-revalidate, no-cache, private
Set-Cookie: bid=G9T75uKFgJ8; Expires=Tue, 05-Nov-19 05:02:06 GMT; Domain=.douban.com; Path=/
X-DOUBAN-NEWBID: G9T75uKFgJ8
X-DAE-Node: anson13
X-DAE-App: book
Server: dae
X-Frame-Options: SAMEORIGIN
Data:
 {"rating":{"max":10,"numRaters":158,"average":"9.3","min":0},"subtitle":"一卷本","pubdate":"1962","image":"https://img3.doubanio.com\/view\/subject\/m\/public\/s3254653.jpg","binding":"精装","images":{"small":"https://img3.doubanio.com\/view\/subject\/s\/public\/s3254653.jpg","large":"https://img3.doubanio.com\/view\/subject\/l\/public\/s3254653.jpg","medium":"https://img3.doubanio.com\/view\/subject\/m\/public\/s3254653.jpg"},"alt":"https:\/\/book.douban.com\/subject\/2224879\/","

`requeset.urlopen(url,data,timeout)`

`requeset.read()`

In [3]:
data = bytes(parse.urlencode({'word': 'hello'}), encoding='utf8')
print(data)

b'word=hello'


In [4]:
response = request.urlopen('http://httpbin.org/post', data=data)
print(response.read().decode('utf-8'))

{
  "args": {}, 
  "data": "", 
  "files": {}, 
  "form": {
    "word": "hello"
  }, 
  "headers": {
    "Accept-Encoding": "identity", 
    "Connection": "close", 
    "Content-Length": "10", 
    "Content-Type": "application/x-www-form-urlencoded", 
    "Host": "httpbin.org", 
    "User-Agent": "Python-urllib/3.7"
  }, 
  "json": null, 
  "origin": "116.22.135.73", 
  "url": "http://httpbin.org/post"
}



In [5]:
response = request.urlopen('http://httpbin.org/get', timeout=1)
print(response.read().decode('utf-8'))

{
  "args": {}, 
  "headers": {
    "Accept-Encoding": "identity", 
    "Connection": "close", 
    "Host": "httpbin.org", 
    "User-Agent": "Python-urllib/3.7"
  }, 
  "origin": "116.22.135.73", 
  "url": "http://httpbin.org/get"
}



In [6]:
try:
    response = request.urlopen('http://httpbin.org/get', timeout=0.1)
    print(response.read().decode('utf-8'))
except error.URLError as e:
    if isinstance(e.reason, socket.timeout):
        print('TIME OUT')

TIME OUT
